<a href="https://colab.research.google.com/github/Kishan-Sinha/Campus_placement_Predictor/blob/main/setting_Up_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing necessary libraries

import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
import joblib

# Load the trained model (update path if needed)

model = joblib.load("/content/sample_data/placement_model.pkl")

# Core prediction logic

def predict_placement (
    Age, Gender, Average_GPA, Attendance_perc, Internship_Done,
    In_Robotics, In_Coding_Club, Knows_Machine_Learning, Knows_Java,
    Knows_Cplusplus, Knows_SQL, Knows_Data_Science, Knows_Python,
    Knows_Web_Development, Extracurricular_Activities,
    internship_domain_choice, Backlogs
                      ):

    # Validate all fields (Error handling)

    all_inputs = [
        Age, Gender, Average_GPA, Attendance_perc, Internship_Done,
        In_Robotics, In_Coding_Club, Knows_Machine_Learning, Knows_Java,
        Knows_Cplusplus, Knows_SQL, Knows_Data_Science, Knows_Python,
        Knows_Web_Development, Extracurricular_Activities,
        internship_domain_choice, Backlogs
                 ]

    if any(i is None for i in all_inputs):

        return "⚠️ Please fill in all fields before submitting.", plt.figure()


    # Transform GPA (log scale)

    AvgGPA = np.log1p(Average_GPA)

    # Bin backlogs into ordinal categories

    backlogs_bin = 0 if Backlogs <= 0 else 1 if Backlogs <= 2 else 2

    # Setting a condition-based encoding for internship domains

    domains = {
        "Consulting": 0, "Core Engineering": 0, "Hardware": 0,
        "Higher Studies": 0, "None": 0, "Research": 0, "Software": 0
              }

    if Internship_Done == 1:

        domains[internship_domain_choice] = 1

    else:

        domains["None"] = 1  # Only None is 1 if internship not done

    # Prepare feature vector

    input_data = np.array([[
        Age, Gender, AvgGPA, Attendance_perc, Internship_Done,
        In_Robotics, In_Coding_Club, Knows_Machine_Learning, Knows_Java,
        Knows_Cplusplus, Knows_SQL, Knows_Data_Science, Knows_Python,
        Knows_Web_Development, Extracurricular_Activities,
        domains["Consulting"], domains["Core Engineering"], domains["Hardware"],
        domains["Higher Studies"], domains["None"], domains["Research"], domains["Software"],
        backlogs_bin
                          ]])

    # Predict probabilities and output

    prob = model.predict_proba(input_data)[0]
    pred = model.predict(input_data)[0]
    margin = (prob[1] - prob[0]) * 100  #in percentage

    # Responding with positive outputs based on different margins

    if margin <= -20:
        result_str = "💡 Don’t lose hope, this is just the beginning. Every expert was once a beginner!"
    elif -20 < margin <= -10:
        result_str = "📉 You're facing challenges, but they’re temporary. Keep pushing!"
    elif -10 < margin <= 0:
        result_str = "🔄 You're very close. A bit more effort can change the outcome!"
    elif 0 < margin < 10:
        result_str = "🤝 Postive placement possibility! A few strategic improvements could make you more placement-ready."
    elif 10 <= margin < 20:
        result_str = "💪 Great momentum! You're on the right track. Keep going."
    else:
        result_str = "🎯 Excellent chances of placement! Stay consistent!"

    # Create bar plot for visual representation

    fig, ax = plt.subplots()
    bars = ax.bar(['Not Placed', 'Placed'], prob, color=['orange', 'green'])
    ax.set_ylim(0, 1)
    ax.set_ylabel("Probability")
    ax.set_title("Placement Probability")
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2.0, height, f'{int(height * 100)}%', ha='center', va='bottom')

    return result_str, fig

# Show or hide "Internship domain selector" based on "internship done" status

def update_domain_visibility(internship_done):

    return gr.update(visible=bool(internship_done), value="None")

# --- Interface UI Setup ---

with gr.Blocks() as demo:

    gr.Markdown("## 🎓 PLACEMENT PREDICTION MODEL")

    with gr.Row():

        with gr.Column(scale=1):

            # 1st. Basic inputs
            age = gr.Number(label="Age")
            gender = gr.Radio(choices=[("Male", 1), ("Female", 0)], label="Gender")
            average_gpa = gr.Number(label="Average GPA")
            attendance_perc = gr.Number(label="Attendance Percentage (%) (0 - 100)")
            internship_done = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="Any Internship Done ?")

            # 2nd. Skills & Activities
            in_robotics = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="Interested in Robotics ?")
            in_coding_club = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="In Any Coding Club ?")
            knows_machine_learning = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="Know Machine Learning ?")
            knows_java = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="Know Java ?")
            knows_cplusplus = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="Know C++ ?")
            knows_sql = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="Know SQL ?")
            knows_data_science = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="Know Data Science ?")
            knows_python = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="Know Python ?")
            knows_web_development = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="Know Web Development ?")
            extracurricular_activities = gr.Radio(choices=[("Yes", 1), ("No", 0)], label="Involved in Extracurricular Activities ?")

            # 4th. Internship domain (shown only if Internship Done = Yes)
            internship_domain_radio = gr.Radio(
                choices=["Consulting", "Core Engineering", "Hardware", "Higher Studies", "Research", "Software", "None"],
                label="Select Internship Domain",
                visible=False
                                              )

            # 5th. Academic backlog input
            backlogs = gr.Number(label="Backlogs")

            # Submit button

            predict_btn = gr.Button("🚀 Predict Placement")

        with gr.Column(scale=1):
            output_text = gr.Textbox(label="Feedback")
            output_plot = gr.Plot(label="Placement Probability")

    # Show / hide domain selection based on internship status

    internship_done.change(
        fn=update_domain_visibility,
        inputs=internship_done,
        outputs=internship_domain_radio
                          )

    # Final click connection  that displays the specified outputs with respect to given inputs

    predict_btn.click(
        fn=predict_placement,
        inputs=[
            age, gender, average_gpa, attendance_perc, internship_done,
            in_robotics, in_coding_club, knows_machine_learning, knows_java,
            knows_cplusplus, knows_sql, knows_data_science, knows_python,
            knows_web_development, extracurricular_activities,
            internship_domain_radio, backlogs
        ],
        outputs=[output_text, output_plot]
    )

# Launch the interface

demo.launch(share=True)


**The above code will launch a temporary link that will transport us to the UI for accessing the model**